# Zillow Zestimate Overview

In [2]:
import pandas as pd
import gc

from bokeh.charts import Histogram, Bar, BoxPlot, Scatter, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot

output_notebook()

Loading BokehJS ...

## Train dataset (short)

Short means that only 3 columns are present (parcelid, logerror, tansactiondate)

In [3]:
train_2016_short = pd.read_csv("../data/train_2016.csv", parse_dates=["transactiondate"])
print(train_2016_short.shape)
print("from:", train_2016_short["transactiondate"].min())
print("to:  ", train_2016_short["transactiondate"].max())
train_2016_short.head()

(90811, 3)
from: 2016-01-01 00:00:00
to:   2016-12-30 00:00:00


,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


### <font color="blue">logerror</font>

In [ ]:
h = Histogram(train_2016_short["logerror"], title="logerror distribution", tools=["xwheel_zoom"], width=900, height=350)

train_2016_short["label"] = "0"
p = BoxPlot(train_2016_short, label="label", values='logerror', title="logerror dispersion", 
            tools=["ypan", "ywheel_zoom", "save"], width=700, height=350)

show(gridplot([[h], [p]]))

del train_2016_short["label"]
del h
del p
gc.collect()

In [ ]:
train_2016_logerror_0 = train_2016_short[train_2016_short["logerror"] == 0]
logerror_percent_0 = str(round(100 * len(train_2016_logerror_0)/ float(len(train_2016_short)), 4))
print(logerror_percent_0 + "% of ZERO error")

### <font color="blue">transactiondate</font>

In [ ]:
months = train_2016_short["transactiondate"].astype(str).str.slice(0,7)

time_freqs = pd.DataFrame(months.value_counts())

p = Bar(time_freqs, values='transactiondate', title="monthly transactiondate distribution", legend=False,
        width=800, height=350)
show(p)

del months
del time_freqs
del p
gc.collect()

### <font color="blue">parcelid</font>

duplicated parcelids

In [ ]:
is_duplicated = train_2016_short["parcelid"].duplicated()
duplicated_ids = train_2016_short[is_duplicated]["parcelid"]
duplicateds = train_2016_short[train_2016_short["parcelid"].isin(duplicated_ids)].sort_values(["parcelid"])

duplicateds.head(10)

h = Histogram(duplicateds["logerror"], title="logerror distribution for resold properties", tools=["xwheel_zoom"], width=900, height=350)
show(h)

del h
gc.collect()

Bought and sold again

## Properties dataset

Train data

In [4]:
prop_2016 = pd.read_csv("../data/properties_2016.csv", low_memory=False)
print(prop_2016.shape)
#print(prop_2016.columns)
prop_2016.head()

(2985217, 58)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [4]:
print(len(prop_2016["parcelid"].drop_duplicates()))
print(len(train_2016_short["parcelid"].drop_duplicates()))

2985217
90682


Completing train dataset with attributes from properties

In [5]:
train_complete_2016 = train_2016_short.merge(prop_2016, on="parcelid", how="left")
train_complete_2016.shape

(90811, 60)

Inserting information about the signal of the logerror

In [6]:
train_complete_2016["error_direction"] = 0

positive_index = train_complete_2016[train_complete_2016["logerror"] > 0].index
train_complete_2016.loc[positive_index, "error_direction"] = 1

positive_index = train_complete_2016[train_complete_2016["logerror"] < 0].index
train_complete_2016.loc[positive_index, "error_direction"] = -1

Plotting the location of the properties, the color represents the signal of the logerror.<br>
It seems difficult to have an insight even if zooming in.<br>

In [ ]:
p = Scatter(train_complete_2016, x='longitude', y='latitude', color="error_direction",
            title="Geographic scatter plot",
            xlabel="longitude", ylabel="latitude",
            width=900, height=600)

show(p)

train_complete_2016["error_direction"].value_counts()

#del p
#gc.collect()

In [ ]:
train_complete_2016["latitude"].isnull().sum(), train_complete_2016["longitude"].isnull().sum()

Generating column with the logerror mean for the k neighbors since it was difficult to check it visually with the above plot.

In [7]:
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

def fit_knn(df, k):
    df = df.dropna(subset=["latitude", "longitude"])
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(df[["latitude", "longitude"]])
    return nbrs
    
def k_neighbors_logerror_mean(df_train, df_test, nbrs):        
    logerror_means = []
    distance_means = []
    df_train = df_train.dropna(subset=["latitude", "longitude"])
    df_test = df_test.dropna(subset=["latitude", "longitude"])
    
    for index, row in df_test.iterrows():
        distances, indexes = nbrs.kneighbors(row[["latitude", "longitude"]])
        distances = list(distances[0])
        indexes = list(indexes[0])
        #sometimes there are more than k neighbors with the same location
        if index in indexes:
            indexes.remove(index)
            distances.remove(0)
        
        neighbors = df_train.iloc[indexes]
        neighbors_logerror_mean = neighbors["logerror"].mean()
        neighbors_distance_mean = float(sum(distances)) / len(distances)
        
        logerror_means.append(neighbors_logerror_mean)
        distance_means.append(neighbors_distance_mean)
        
    df_test["nbrs_logerror_mean"] = logerror_means
    df_test["nbrs_distance_mean"] = distance_means
    
    return df_test

nbrs = fit_knn(train_complete_2016, 2)
train_complete_2016 = k_neighbors_logerror_mean(train_complete_2016, train_complete_2016, nbrs)
gc.collect()

740

Checking the **nbrs_logerror_mean** correlation with **logerror**

In [8]:
print("spearman:", train_complete_2016["logerror"].corr(train_complete_2016["nbrs_logerror_mean"], method="spearman"))
print("pearson :", train_complete_2016["logerror"].corr(train_complete_2016["nbrs_logerror_mean"], method="pearson"))

spearman: 0.643235761132
pearson : 0.687486975183


Checking for correlation between absolute values from **nbrs_logerror_mean** and **logerror**

In [10]:
print("spearman:", abs(train_complete_2016["logerror"]).corr(abs(train_complete_2016["nbrs_logerror_mean"]), method="spearman"))
print("pearson: ", abs(train_complete_2016["logerror"]).corr(abs(train_complete_2016["nbrs_logerror_mean"]), method="pearson"))

spearman: 0.505480964238
pearson:  0.670493796109


All numeric columns correlations

In [11]:
corr_dicts = []
for col in train_complete_2016.columns:
    try:
        corr_dicts.append({"column": col, "corr": train_complete_2016[col].astype('float64').corr(train_complete_2016["logerror"])})
    except (TypeError, ValueError):
        pass
corr_df = pd.DataFrame(corr_dicts).sort_values(by="corr", ascending=False) 
corr_df

,column,corr
1,logerror,1.000000
57,nbrs_logerror_mean,0.687487
56,error_direction,0.421615
4,basementsqft,0.253395
45,yardbuildingsqft26,0.086179
13,finishedsquarefeet12,0.041922
12,calculatedfinishedsquarefeet,0.038784
17,finishedsquarefeet6,0.036847
14,finishedsquarefeet13,0.034715
41,threequarterbathnbr,0.034069


In [12]:
p = Scatter(train_complete_2016, x='logerror', y='nbrs_logerror_mean',
            title="logerror by nbrs_logerror_mean scatter plot",
            xlabel="logerror", ylabel="nbrs_logerror_mean",
            width=900, height=600)

show(p)

In [14]:
higests_corr_columns = corr_df[corr_df["corr"] > 0.15].column.tolist()
higests_corr_columns.remove("error_direction")
higests_corr_columns = ["parcelid"] + higests_corr_columns

train = train_complete_2016[higests_corr_columns]
train.to_csv("/home/tales/dev/projects/zillow-zestimate/train-1.csv", index=False)

In [1]:
#higests_corr_columns.remove("logerror")
test = k_neighbors_logerror_mean(train_complete_2016, prop_2016, nbrs)[higests_corr_columns]
test.to_csv("/home/tales/dev/projects/zillow-zestimate/test-1.csv", index=False)

NameError: name 'k_neighbors_logerror_mean' is not defined

In [18]:
test.merge(train, on="parcelid", how="inner")
gc.collect()

4133

In [ ]:
prop_2016_no_na = prop_2016[["latitude", "longitude"]].dropna()
prop_2016_no_na.head()
len(prop_2016_no_na), len(prop_2016), prop_2016_no_na.index.max(), prop_2016.index.max()

In [ ]:
prop_2016_no_na[["latitude", "longitude"]].tail()